# <u>Análisis de Regresión y Regularizaciones</u>

### Librerías

In [ ]:
# Import necessary libs

import pandas as pd
import numpy as np
from sklearn import preprocessing
from math import sqrt

import warnings
warnings.filterwarnings('ignore')
from scipy import stats

## Modelos de regresion
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm           
import statsmodels.formula.api as smf  

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

#Metricas de evaluación
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score

#separar train and test
from sklearn.model_selection import train_test_split

# estadística y matemáticas
#import pandas as pd
import scipy.stats as scy
from scipy.stats import kurtosis

### Datos del Negocio

In [ ]:
dataset = pd.read_csv('data/demanda_data1.csv',sep=',',parse_dates = ['tiempo'])
dataset

## 1. Análisis Exploratorio de los Datos (EDA)

In [ ]:
dataset.describe()

In [ ]:
# Histograma del Target
sns.set()
plt.figure()
sns.distplot(dataset["UnidadesDemandas"] , label="UnidadesDemandas")
plt.legend()
plt.show()

In [ ]:
# Grouped boxplot
plt.figure()
sns.boxplot(x='UnidadesDemandas', data=dataset);

## 2. Correlaciones lineales

In [ ]:
dataset.corr(method='pearson')

In [ ]:
sns.set(font_scale=2)
corr_matrix = dataset.corr()
plt.figure(figsize=(16, 10))
ax = sns.heatmap(corr_matrix,annot=True, fmt=".1f",cmap="YlGnBu") 
ax.set_ylim(sorted(ax.get_xlim(), reverse=True))

### Distribución de las variables cuantitativas 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(14, 12))
axes = axes.flat
columnas_object = dataset.select_dtypes(include=['float']).columns

for i, colum in enumerate(columnas_object):
    dataset[colum].value_counts().plot.hist(ax = axes[i], bins=30)
    axes[i].set_title(colum, fontsize = 12, fontweight = "bold")
    axes[i].tick_params(labelsize = 4)
    axes[i].set_xlabel("")

fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Distribución variables numericas',
             fontsize = 18, fontweight = "bold");

### Gráfico para cada variable cualitativa

In [ ]:
for feature in ['Marca']:
    var = dataset.groupby(feature)[feature].count().sort_values(ascending = False)
    fig = plt.figure()
    ax1 = fig.add_subplot(1,1,1)
    ax1.set_xlabel(feature)
    ax1.set_ylabel('Cantidad')
    ax1.set_title(feature)
    var.plot(kind='bar')
    plt.show()

### Preprocesing de la data cualitativa

In [ ]:
data= pd.get_dummies(dataset, columns=['Marca']) 
data.head()

## 3.Modelos de Regresión

### Selección de muestras de entrenamiento y validación 

In [ ]:
X_train = data[dataset.tiempo < '2014-01-01'].drop(['UnidadesDemandas','tiempo'], axis=1)
X_test = data[dataset.tiempo >= '2014-01-01'].drop(['UnidadesDemandas','tiempo'], axis=1)
y_train = data[dataset.tiempo < '2014-01-01'][['UnidadesDemandas']]
y_test= data[dataset.tiempo >= '2014-01-01'][['UnidadesDemandas']]

In [ ]:
X_train.shape, X_test.shape

### Segundo método con split aleatorio

In [ ]:
#X = dataset.drop(['UnidadesDemandas','tiempo'], axis=1)
#y = dataset[['UnidadesDemandas']]

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

## 3.1.Modelo de Regresión Lineal Múltiple (usando librería statsmodels)

In [ ]:
M_R = sm.OLS(y_train,X_train).fit()
M_R.summary()

#### Hallazgos:
* El p valor de la prueba ANOVA nos valida que el modelo de regresión es válido.
* De encontrar variables que no cumplen la hipótesis individual (prueba t) hay que quitarlos y volver a estimar el modelo de regresión.
* Variables como `CantOfertas` y `TieneUbicacionEspecifica` no cumplen con la validez individual, deberían excluirse del modelo, veremos como se desempeñan estas variables con las técnicas regularizadas.

In [ ]:
# Predicción del test
y_pred = M_R.predict(X_test)

In [ ]:
# Evaluación del modelo
print('Mean Absolute Error:',      metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:',       metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:',  np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

rmse_ols = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"El error (rmse) de test es: {rmse_ols}")

Las predicciones del modelo final se alejan en promedio 42.321 unidades del valor real

In [ ]:
# R2 del modelo 
r2_ols = np.sqrt(r2_score(y_test,y_pred))
print("")
print(f"El R2 de test es: {r2_ols}")

### Analisis Residual y Supuestos 

In [ ]:
# Valores predecidos vs valores reales (Linealidad)
plt.scatter(y_test, y_pred, edgecolors=(0, 0, 0), alpha = 0.4)
plt.plot([y_test.min(), y_test.max()], 
         [y_test.min(), y_test.max()],'k--', color = 'black', lw=2)
plt.title('Valor predicho vs valor real', fontsize = 20, fontweight = "bold")
plt.xlabel('Real')
plt.ylabel('Predicción')

In [ ]:
## Prueba de normalidad 
from scipy.stats import shapiro
stat, p = shapiro(y_pred)
print('Statistics=%.3f, p=%.3f' % (stat, p))

In [ ]:
# Normalidad
p = sns.distplot(y_pred,kde=True)
p = plt.title('Normalidad de los Residuales')
plt.show()

La multicolinealidad en el análisis de regresión ocurre cuando dos o más variables explicativas están altamente correlacionadas entre sí, de manera que no brindan información única o independiente en el modelo de regresión. Si el grado de correlación entre variables es lo suficientemente alto, puede causar problemas al ajustar e interpretar el modelo de regresión.

Una forma de detectar la multicolinealidad es mediante el uso de una métrica conocida como factor de inflación de la varianza (VIF) , que mide la correlación y la fuerza de la correlación entre las variables explicativas en un modelo de regresión .

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

#calcular VIF para cada variable explicativa
vif = pd.DataFrame ()
vif ['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range (X_train.shape[1])]
vif ['variable'] = X_train.columns

#ver VIF para cada variable explicativa 
vif

El valor de VIF comienza en 1 y no tiene límite superior. Una regla general para interpretar los VIF es la siguiente:

* Un valor de 1 indica que no hay correlación entre una variable explicativa dada y cualquier otra variable explicativa en el modelo.
* Un valor entre 1 y 5 indica una correlación moderada entre una variable explicativa dada y otras variables explicativas en el modelo, pero esto a menudo no es lo suficientemente grave como para requerir atención.
* Un valor mayor que 5 indica una correlación potencialmente severa entre una variable explicativa dada y otras variables explicativas en el modelo. En este caso, las estimaciones de los coeficientes y los valores p en el resultado de la regresión probablemente no sean confiables.

In [ ]:
# Homocedastidiad es decir errores uniformes en sus residuos
plt.scatter(list(range(len(y_test))), 
           y_test.index.values - np.array(y_pred), edgecolors=(0, 0, 0), alpha = 0.4)
plt.axhline(y = 3400, linestyle = '--', color = 'black', lw=2)
plt.title('Residuos del modelo', fontsize = 20, fontweight = "bold")
plt.xlabel('id')
plt.ylabel('Residuo')

In [ ]:
# Podemos crear nuestra metodología de medición de regresores!

df_test = pd.concat([X_test,y_test],axis=1)

df_test['y_pred_lineal'] = y_pred

In [ ]:
df_test

## 4.Modelos lineales con Regularización

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

## 4.1 Regularización Ridge

In [ ]:
# Creación y entrenamiento del modelo (con búsqueda por CV del valor óptimo alpha)
# Por defecto RidgeCV utiliza el mean squared error
modelo_RG = RidgeCV(
            alphas          = [1000,800,600,400,200,100,80,60,30,15,10,9,8,7,6,5,1,0.1,0.01,0.005,0.0001,0.00005],
            fit_intercept   = True,
            normalize       = True,
            store_cv_values = True
         )

_ = modelo_RG.fit(X = X_train, y = y_train)

Cuando se utiliza regularización, es útil evaluar cómo se aproximan a cero los coeficientes a medida que se incrementa el valor de alpha así como la evolución del error de la optimización en función del alpha empleado.

In [ ]:
modelo_RG

In [ ]:
# Evolución de los coeficientes en función de alpha
alphas = modelo_RG.alphas
coefs = []

for alpha in alphas:
    modelo_temp = Ridge(alpha=alpha, fit_intercept=True, normalize=True)
    modelo_temp.fit(X_train, y_train)
    coefs.append(modelo_temp.coef_.flatten())

fig, ax = plt.subplots()
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_xlabel('alpha')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo en función de la regularización');
plt.axis('tight')
plt.show()

 A medida que aumenta el valor de alpha, la regularización es mayor y el valor de los coeficientes se reduce

In [ ]:
# el valor de alpha es:
modelo_RG.alpha_

In [ ]:
# Mejor modelo alpha óptimo + 1sd
modelo_RG = Ridge(alpha=0.1, normalize=True)
modelo_RG.fit(X_train, y_train)

In [ ]:
# Coeficientes del modelo
# ==============================================================================
df_coeficientes = pd.DataFrame(
                        {'predictor': X_train.columns,
                         'coef': modelo_RG.coef_.flatten()})

fig, ax = plt.subplots(figsize=(10, 8))
ax.stem(df_coeficientes.predictor, df_coeficientes.coef, markerfmt=' ')
plt.xticks(rotation=90, ha='right', size=10)
ax.set_xlabel('variable')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo');

In [ ]:
# Pintamos los coeficientes
print(pd.Series(modelo_RG.coef_.ravel(), index = X_train.columns).sort_values(ascending = False))

In [ ]:
# Predicciones del test
predicciones = modelo_RG.predict(X=X_test)
predicciones = predicciones.flatten()
predicciones[:10]

In [ ]:
# Error de test del modelo 
rmse_ridge = np.sqrt(metrics.mean_squared_error(y_test, predicciones))
print("")
print(f"El ECM de test es: {rmse_ridge}")

Las predicciones del modelo final se alejan en promedio 42.313 unidades del valor real.

* Valores altos de alpha producen modelo que tienden a ajustar los pesos a cero. Es decir aumenta el sesgo del modelo

* Valores bajos de alpha lo que produce es que el modelo se comporta como una regresión lineal sin regularización

En ese sentido es muy importante elegir el valor optimo de alpha

In [ ]:
# R2 del modelo 
r2_ridge = np.sqrt(r2_score(y_test,predicciones))
print("")
print(f"El error (rmse) de test es: {r2_ridge}")

## 4.2 Regularización Lasso

In [ ]:
# Creación y entrenamiento del modelo (con búsqueda por CV del valor óptimo alpha)
# Por defecto LassoCV utiliza el mean squared error
modelo_LS = LassoCV(
            alphas          = [1000,800,600,400,200,100,80,60,30,15,10,9,8,7,6,5,1,0.1,0.01,0.001],
            normalize       = True,
            cv              = 10
         )
_ = modelo_LS.fit(X = X_train, y = y_train)

In [ ]:
# Evolución de los coeficientes en función de alpha
alphas = modelo_LS.alphas_
coefs = []

for alpha in alphas:
    modelo_temp = Lasso(alpha=alpha, fit_intercept=False, normalize=True)
    modelo_temp.fit(X_train, y_train)
    coefs.append(modelo_temp.coef_.flatten())

fig, ax = plt.subplots()
ax.plot(alphas, coefs)
ax.set_xscale('log')
ax.set_ylim([-15,None])
ax.set_xlabel('alpha')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo en función de la regularización');

A medida que aumenta el valor de alpha, la regularización es mayor y más predictores quedan excluidos (su coeficiente es 0).

In [ ]:
# Número de predictores incluidos (coeficiente !=0) en función de alpha
alphas = modelo_LS.alphas_
n_predictores = []

for alpha in alphas:
    modelo_temp = Lasso(alpha=alpha, fit_intercept=True, normalize=True)
    modelo_temp.fit(X_train, y_train)
    coef_no_cero = np.sum(modelo_temp.coef_.flatten() != 0)
    n_predictores.append(coef_no_cero)

fig, ax = plt.subplots()
ax.plot(alphas, n_predictores)
ax.set_xscale('log')
ax.set_ylim([-15,None])
ax.set_xlabel('alpha')
ax.set_ylabel('nº predictores')
ax.set_title('Predictores incluidos en función de la regularización');

In [ ]:
# el valor de alpha es:
modelo_LS.alpha_

In [ ]:
# Mejor modelo alpha óptimo + 1sd
modelo_LS = Lasso(alpha=0.1, normalize=True)
modelo_LS.fit(X_train, y_train)

In [ ]:
# Coeficientes del modelo
df_coeficientes = pd.DataFrame(
                        {'predictor': X_train.columns,
                         'coef': modelo_LS.coef_.flatten()}
                  )

# Predictores incluidos en el modelo (coeficiente != 0)
df_coeficientes[df_coeficientes.coef != 0]

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3.84))
ax.stem(df_coeficientes.predictor, df_coeficientes.coef, markerfmt=' ')
plt.xticks(rotation=90, ha='right', size=10)
ax.set_xlabel('variable')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo');

In [ ]:
# Pintamos los coeficientes
print(pd.Series(modelo_LS.coef_, index = X_train.columns).sort_values(ascending = False))

In [ ]:
# Predicciones test
predicciones = modelo_LS.predict(X=X_test)
predicciones = predicciones.flatten()
predicciones[:10]

In [ ]:
# Error de test del modelo 
rmse_lasso = np.sqrt(metrics.mean_squared_error(y_test, predicciones))
print("")
print(f"El error (rmse) de test es: {rmse_lasso}")

Las predicciones del modelo final se alejan en promedio 44.394 unidades del valor real, utilizando solo 8 variables del total de predictores.

In [ ]:
# R2 del modelo 
r2_lasso = np.sqrt(r2_score(y_test,predicciones))
print("")
print(f"El R2 de test es: {r2_lasso}")

## 4.3 Regularización Elastic net

In [ ]:
# Creación y entrenamiento del modelo (con búsqueda por CV del valor óptimo alpha)
# Por defecto ElasticNetCV utiliza el mean squared error
modelo_EL = ElasticNetCV(
            l1_ratio        = [0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99],
            alphas          = [1000,800,600,400,200,100,80,60,30,15,10,9,8,7,6,5,1,0.1,0.01,0.001],
            normalize       = True,
            cv              = 10
         )
_ = modelo_EL.fit(X = X_train, y = y_train)

In [ ]:
modelo_EL

In [ ]:
modelo_EL.l1_ratio_  #ratio de ridge

In [ ]:
modelo_EL.alpha_

In [ ]:
# Coeficientes del modelo
df_coeficientes = pd.DataFrame(
                        {'predictor': X_train.columns,
                         'coef': modelo_EL.coef_.flatten()}
                  )

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3.84))
ax.stem(df_coeficientes.predictor, df_coeficientes.coef, markerfmt=' ')
plt.xticks(rotation=90, ha='right', size=10)
ax.set_xlabel('variable')
ax.set_ylabel('coeficientes')
ax.set_title('Coeficientes del modelo');

In [ ]:
# Pintamos los coeficientes
print(pd.Series(modelo_EL.coef_, index = X_train.columns).sort_values(ascending = False))

In [ ]:
# Predicciones test
predicciones = modelo_EL.predict(X=X_test)
predicciones = predicciones.flatten()

In [ ]:
# Error de test del modelo 
rmse_elastic = np.sqrt(mean_squared_error(y_test,predicciones))
print("")
print(f"El error (rmse) de test es: {rmse_elastic}")

Las predicciones del modelo final se alejan en promedio 42.088 unidades del valor real

In [ ]:
# R2 del modelo 
r2_elastic = np.sqrt(r2_score(y_test,predicciones))
print("")
print(f"El R2 de test es: {r2_elastic}")

### COMPARACION DEL MEJOR MODELO EN BASE AL RMSE

In [ ]:
# Comparamos los 4 modelos
df_comparacion1 = pd.DataFrame({
                    'modelo': ['OLS', 'Ridge', 'Lasso', 'Elastic-net'],
                    'test rmse': [rmse_ols, rmse_ridge, rmse_lasso, rmse_elastic]
                 })
ax = df_comparacion1.set_index('modelo').plot(kind="barh",title="Comparación de Modelos con RMSE TEST")
ax.set_ylabel("RMSE", fontsize="large")
ax.get_legend().remove()

In [ ]:
df_comparacion1

In [ ]:
# Comparamos los 4 modelos
df_comparacion2 = pd.DataFrame({
                    'modelo': ['OLS', 'Ridge', 'Lasso', 'Elastic-net'],
                    'test rmse': [r2_ols, r2_ridge, r2_lasso, r2_elastic]
                 })
ax = df_comparacion2.set_index('modelo').plot(kind="barh",title="Comparación de Modelos con R2 TEST")
ax.set_ylabel("R2", fontsize="large")
ax.get_legend().remove()

In [ ]:
df_comparacion2

Conclusión: Ridge tine mas explicativas por eso r2 mayor, pero Lasso nos da una regresión con las mejores variables.